In [ ]:
import os
import csv
import glob
import sys
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import FileLink

In [ ]:
SOURCES_TEXT = '''<p>
<b>Land-Ocean Temperature Index</b>
<a href="https://climate.nasa.gov/vital-signs/global-temperature/"
target="_blank">Global Temperature (NASA)</a>
,
<a href="https://data.giss.nasa.gov/gistemp/"
target="_blank">GISS Surface Temperature Analysis (NASA)</a>
</p><p>
This site is based on data downloaded from the following site on 2020-07-14:
<a href="https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt"  # noqa
target="_blank">Global Mean Estimates based on Land and Ocean Data (NASA)</a>
<br>
'''

sources = widgets.HTML(value = SOURCES_TEXT)
sources

In [ ]:
def upload_df_from_file(path):
    return pd.read_csv(path, escapechar='#')

In [ ]:
UPLOAD_PATH = 'data/land-ocean-temp-index.csv'
df = upload_df_from_file(UPLOAD_PATH)

In [ ]:
df_output = widgets.Output()

In [ ]:
#df_output

In [ ]:
# right click on df_output
# select "Create New View for Output"

In [ ]:
def display_df(df):
    df_output.clear_output(wait=True)
    with df_output: 
        display(df)
        
#display_df(df)

In [ ]:
plot_output = widgets.Output()

In [ ]:
# plot_output

In [ ]:
# again, "Create New View for Output"

In [ ]:
def display_plot(df):
    plot_output.clear_output(wait=True)
    with plot_output:
        plt.xlabel('Year')
        plt.ylabel('Temperature')
        plt.plot(df['Year'], df['Temperature'])
        if 'EMA' in df:
            plt.plot(df['Year'], df['EMA'])
        plt.show()

In [ ]:
#display_plot(df)

In [ ]:
def add_or_update_EMA_column(df, ema_span=2):
    df['EMA'] = df['Temperature'].ewm(span=ema_span, adjust=False).mean()

In [ ]:
add_or_update_EMA_column(df)

In [ ]:
main_hbox = widgets.HBox(children = (df_output, plot_output))
display_df(df)
display_plot(df)
main_hbox

In [ ]:
def on_ema_span_change(change):
    add_or_update_EMA_column(df, change['new'])
    display_df(df)
    display_plot(df)
    
ema_slider = widgets.IntSlider(min=1, max=7, description='EMA Span')
ema_slider.observe(on_ema_span_change, names='value')

In [ ]:
#ema_slider

In [ ]:
DOWNLOAD_PATH = 'data/output_data.csv'

def create_output_file(df, path):
    """Prep data for export."""

    df.to_csv(path, index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# Reproducibility concerns. How to track what
# parameters where used to create the dataset.
# Filenames?

In [ ]:
download_button = widgets.Button(description = 'Download', button_style='info')
file_link_output = widgets.Output()
download_hbox = widgets.HBox(children = (ema_slider, download_button, file_link_output))
download_hbox

In [ ]:
# NOTE: if you click the link here, the file will 
# open up in a JupyterLab. If you try and use the same
# link in the Voila Preview, it won't work because JupyterLab
# isn't configured with the settings to allow it. However
# if you are running the web app outside JupyterLab with 
# Voila, you can whitelist any filename with a configuration
# setting to allow for download.
#
# $ voila data_dashboard.ipynb --VoilaConfiguration.file_whitelist="['*.csv']"
# 
# For more, see https://github.com/voila-dashboards/voila/issues/560

In [ ]:
def on_download_button_click(b):
    create_output_file(df, DOWNLOAD_PATH)
    file_link_output.clear_output(wait=True)
    with file_link_output:
        display(FileLink(DOWNLOAD_PATH, result_html_prefix="Click here to save file: "))
        
download_button.on_click(on_download_button_click)

In [ ]:
# We are already starting to see how our development
# could benefit from literate programming. It would be
# nice it we could leave in visualizations that just 
# get ignored by the final product. This is one of 
# the benefits of Panel, which is more "literate" by 
# design. ipywidgets doesn't have this feature. 
# We can us the Output View to help with this, but 

In [ ]:
# What is good about this approach

# 1. I didn't take long because we used code we already had
# 2. It works

# What we could improve about the approach
# 1. We have to delete or copy-paste cells that we used in intermediate steps
# 2. We have to leave our comments as just that - comments
#    in code cells instead of markdown
# 3. The Output widgets flash when EMA Span changes. Whats
#    worse is that the 

# What approaches could we take from here?
# 1. We could start offloading our code into Python files
#    but that still doesn't avoid the problem of copy-pasting
#    because we still want to see how visual components work 
#    in intermediate steps in the notebook
# 2. 